In [ ]:
import pyspark

In [23]:
import findspark
findspark.init()
import os
os.environ["JAVA_HOME"] = "C:\\Java\\jdk1.8.0_191"
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover

In [8]:
import pandas as pd
review_data = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\train\\PA_train_yelp_academic_dataset_review.csv', error_bad_lines=False)
user_data = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\train\\PA_train_yelp_academic_dataset_user.csv', error_bad_lines=False)
business_data = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\train\\PA_train_yelp_academic_dataset_business.csv', error_bad_lines=False)
user_review=pd.read_csv("reviews_topic_new.csv")

In [7]:
import pickle

In [8]:
with open('lda_model_20_21.pkl', 'rb') as fp:
    lda_model_20 = pickle.load(fp)

c:\users\ragini\appdata\local\programs\python\python37\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
def get_user_features(review,user_data):
    new_user_review_df=review.join(user_data.set_index('user_id'),on='user_id').drop(['cool','name','friends','elite','yelping_since','review_count','fans','useful','funny'],axis=1)
    return new_user_review_df
    

In [12]:
new_user_review_df=get_user_features(user_review,user_data)

In [13]:
new_user_review_df.to_csv('user_FeatureVec.csv',index=False)

In [3]:
def get_categ(business_data):
    allCategories=set()
    for row in business_data['categories']:
        row_set = set(row.replace('\\','').replace('/','').strip(" ").split(','))
        row_set = set(x.strip() for x in row_set)
        allCategories=allCategories.union(row_set)
    allCategories=sorted(allCategories)
    return allCategories

In [4]:
def getBusinessCategVector(row,allCategories):
    row = str(row)
    categ_row=row.replace('\\','').replace('/','').strip(" ").split(',')
    categ_row =[ x.strip() for x in sorted(categ_row)]
    one_hot_categ = [categ_row.count(x) for x in allCategories]
    return one_hot_categ

In [5]:
def get_business_features(business_data,review,train_business):
    filtered_business_data = business_data.loc[:,['business_id','stars','is_open','categories','attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel','attributes.RestaurantsPriceRange2']]
    #new_business_review_df= user_review.join(business_data.set_index('business_id'),on='business_id')
    filtered_business_data.rename(index=str,columns={'stars':'business_stars'},inplace=True)
    new_business_review_df= review.join(filtered_business_data.set_index('business_id'),on='business_id')
    allCategories = get_categ(train_business)
    business_categ_vector = list(map(lambda x:getBusinessCategVector(x,allCategories),new_business_review_df['categories'].tolist()))
    business_categ_vector_df = pd.DataFrame(business_categ_vector,columns=[x for x in (allCategories)])
    business_reviews_df_categ= pd.concat([new_business_review_df,business_categ_vector_df],axis=1).drop(['categories'],axis=1)
    return business_reviews_df_categ

In [35]:
business_reviews_df_categ = get_business_features(business_data,user_review)

In [37]:
business_reviews_df_categ.to_csv('business_FeatureVec.csv',index=False)

In [2]:
# busi_review_busi_vec_df = pd.concat([categ_vec,business_data_filtered.drop(["categories"],axis=1)],axis=1)
# busi_review_busi_vec_df.rename(index=str,columns={'stars':'business_stars'},inplace=True)
# busi_review_busi_vec_df = user_review_df_load.join(busi_review_busi_vec_df.set_index('business_id'),on='business_id').drop(['text','user_id','stars'],axis=1)
# print(busi_review_busi_vec_df.count())
# busi_review_busi_vec_df.to_csv('busi_review_featureVec.csv',index=False)

In [14]:

review_data_valid = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\valid\\PA_valid_yelp_academic_dataset_review.csv', error_bad_lines=False)
user_data_valid = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\valid\\PA_valid_yelp_academic_dataset_user.csv', error_bad_lines=False)
business_data_valid = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\valid\\PA_valid_yelp_academic_dataset_business.csv', error_bad_lines=False)
review_topic_valid = pd.read_csv("valid/reviews_topic_valid.csv")


In [ ]:
user_review_df_valid=get_user_features(review_topic_valid,user_data_valid)
user_review_df_valid.to_csv('valid/user_FeatureVec_valid.csv',index=False)
business_reviews_df_categ_valid = get_business_features(business_data_valid,review_topic_valid,business_data)
business_reviews_df_categ_valid.to_csv('valid/business_FeatureVec_valid.csv',index=False)

In [10]:
review_data_test = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\test\\PA_test_yelp_academic_dataset_review.csv', error_bad_lines=False)
user_data_test = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\test\\PA_test_yelp_academic_dataset_user.csv', error_bad_lines=False)
business_data_test = pd.read_csv('..\\..\\PA\\Restaurants-new\\Restaurants\\test\\PA_test_yelp_academic_dataset_business.csv', error_bad_lines=False)


In [ ]:
review_topic_test = pd.read_csv("test/reviews_topic_test.csv")
user_review_df_test = get_user_features(review_topic_test,user_data_test)
user_review_df_test.to_csv('test/user_FeatureVec_test.csv',index=False)
business_reviews_df_categ_test = get_business_features(business_data_test,review_topic_test,business_data)
business_reviews_df_categ_test.to_csv('test/business_FeatureVec_test.csv',index=False)

In [43]:
business_reviews_df_categ_test.shape

(65591, 338)

In [60]:
business_reviews_topic_attr=get_business_features(business_data,pd.read_csv('business_reviews_topic.csv'),business_data)
#business_reviews_topic_attr = pd.read_csv('business_reviews_topic.csv').join(business_data.set_index('business_id'),on='business_id')

In [61]:
business_reviews_topic_attr.to_csv('business_reviews_topic_attr.csv',index=False)

In [13]:
business_reviews_topic_attr_test = get_business_features(business_data_test,pd.read_csv('test/business_reviews_topic_test.csv'),business_data)
business_reviews_topic_attr_test.to_csv('test/business_reviews_topic_attr_test.csv',index=False)

In [15]:
business_reviews_topic_attr_valid = get_business_features(business_data_valid,pd.read_csv('valid/business_reviews_topic_valid.csv'),business_data)
business_reviews_topic_attr_valid.to_csv('valid/business_reviews_topic_attr_valid.csv',index=False)